# Forgotten Runes Loracle Langchain RAG Example

One of the most common ways to use The Book of Lore and Wizzypedia with AI is by using RAG: Retrieval-Augmented-Generation. The idea is you search for snippets of the text in a vector database and then pass that context along with the user's chat to GPT and it answers the question.

This code uses langchain and shows a basic way to query the Forgotten Runes Pinecone server and create a mini-Loracle

#### Resources:

- [RAG | 🦜️🔗 Langchain](https://python.langchain.com/docs/expression_language/cookbook/retrieval) 
- [Returning sources | 🦜️🔗 Langchain](https://python.langchain.com/docs/use_cases/question_answering/sources)

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Pinecone
from langchain_core.runnables import  RunnablePassthrough
from langchain.chains.query_constructor.base import AttributeInfo
import pinecone

### Environment Variables

Setup a `.env` file with your own API Keys

In [8]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY")
os.environ["PINECONE_ENV"] = os.getenv("PINECONE_ENV")
pinecone_index_name = "wizzypedia"

## Pinecone Retriever Setup

Here we tell Langchain on how to fetch from Pinecone. The Pinecone database has several other fields, but we only need `text` for now. 

A _Retriever_ has an intermediate LLM step which will take the index definition and the question and then ask the LLM to form the best query. We then query Pinecone and return the matching documents.

Using LangSmith (langchain tracing) makes this easier to understand what is going on, so I recommend you try it out.


In [9]:

metadata_field_info = [
    AttributeInfo(
        name="text",
        description="The detailed text or description of the item",
        type="string",
    ),
]
document_content_description = "A brief document from a character story or wiki page"

# define the index
embeddings = OpenAIEmbeddings()
index = pinecone.Index(pinecone_index_name, host=os.environ["PINECONE_ENV"])
vectorstore = Pinecone.from_existing_index(
    index_name=pinecone_index_name, embedding=embeddings
)

llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True
)

## Basic RAG

In [11]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [12]:
chain.invoke("what color are Kobolds?")

'Kobolds have green skin.'